In [ ]:
#| hide
#| default_exp matches

# Matches

> Data about ISL matches.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import json
import datetime
import logging

import pytz
import requests
import pandas as pd

from indian_super_league.constants import *

In [ ]:
#| export
def fetch_matches():
    url = 'https://www.indiansuperleague.com/default.aspx?methodtype=3&client=3747164737&sport=2&league=india_sl_stats&timezone=0530&language=&tournament=india_sl_stats_2024'
    headers = {
        'accept': '*/*',
        'referer': 'https://www.indiansuperleague.com/',
        'user-agent': 'Mozilla/5.0'
    }
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        logging.info(f"API request successful. Content length: {len(r.content)}")
        return r.text
    logging.error(f"API request failed. Status code: {r.status_code}")
    return None

In [ ]:
#| eval: false
#| output: false
matches_data = fetch_matches()
print(json.dumps(json.loads(matches_data), indent = 4))

In [ ]:
#| export
def save_matches(data, filepath):
    if data:
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, 'a') as f:
            f.write(data + "\n")

In [ ]:
#| eval: false
#| output: false
save_matches(matches_data, filepath = data_dir / 'matches.txt')

In [ ]:
#| export
def get_live_matches(filepath, now = datetime.datetime.now(pytz.timezone('Asia/Kolkata'))):
    with open(filepath, 'r') as f:
        logs = f.readlines()

    latest = logs[-1]
    matches = json.loads(latest)['matches']

    matches_df = []
    for match in matches:
        matches_df.append({
            'start_at': match['start_date'],
            'end_at': match['end_date'],
            'game_id': match['game_id'],
        })

    matches_df = pd.DataFrame(matches_df)
    matches_df['start_at'] = pd.to_datetime(matches_df['start_at'])
    matches_df['end_at'] = pd.to_datetime(matches_df['end_at'])

    live_matches = matches_df[
        (now >= matches_df["start_at"] - datetime.timedelta(minutes = 15)) &
        (now <= matches_df["end_at"] + datetime.timedelta(minutes = 30))
    ]
    game_ids = list(live_matches['game_id'].values)
    return(game_ids)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()